In [1]:
%reset -f
import pandas as pd
import numpy as np
seed = 2018
np.random.seed(seed)
from wavhandler import *
import soundfile as sf
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import math
import warnings
from keras.applications.densenet import DenseNet121
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.utils import np_utils

sns.set()
%matplotlib inline

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


# Loading data

In [2]:
current_model = DenseNet121

model_name = 'my_Wingbeats' + current_model.__name__
top_weights_path = TEMP_DATADIR + 'model_' + str(model_name) + '.h5'
logfile = TEMP_DATADIR + 'model_' + str(model_name) + '.log'
batch_size = 32
monitor = 'val_acc'
input_shape = (129, 120, 1)
es_patience = 7
rlr_patience = 3

In [3]:
# data1 = Dataset('increasing dataset')
# data1.target_classes = [i for i in data1.target_classes if "aedes" not in i.split('_')]
# data1.load(only_names=True, text_labels=True)
data2 = Dataset('Wingbeats')
data2.load(only_names=True, nr_signals=np.inf, text_labels=True);
# data3 = Dataset('LG')
# data3.load(only_names=True, text_labels=True);

100%|██████████| 6/6 [00:01<00:00,  3.72it/s]


In [4]:
X_names = data2.filenames #+ data1.filenames #+ data3.filenames
y = data2.y #+ data1.y #+ data3.y
target_names = np.unique(y)
print("Names of all classes: \n{}".format(target_names))

Names of all classes: 
['Ae. aegypti' 'Ae. albopictus' 'An. arabiensis' 'An. gambiae'
 'C. pipiens' 'C. quinquefasciatus']


# Train/Val generators

In [5]:
def train_generator(X_train, y_train, batch_size, target_names):
    while True:
        for start in range(0, len(X_train), batch_size):
            x_batch = []
            y_batch = []
            
            end = min(start + batch_size, len(X_train))
            train_batch = X_train[start:end]
            labels_batch = y_train[start:end]
            
            for i in range(len(train_batch)):
                data, rate = librosa.load(train_batch[i], sr = SR)
                if 'increasing dataset' in train_batch[i].split('/'):
                    data = crop_rec(data)

#                 data = random_data_shift(data, u = .2)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    data = librosa.amplitude_to_db(data)

                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            
            y_batch = np_utils.to_categorical(y_batch, len(target_names))
            
            yield x_batch, y_batch

def valid_generator(X_val, y_val, batch_size, target_names):
    while True:
        for start in range(0, len(X_val), batch_size):
            x_batch = []
            y_batch = []

            end = min(start + batch_size, len(X_val))
            test_batch = X_val[start:end]
            labels_batch = y_val[start:end]

            for i in range(len(test_batch)):
                data, rate = librosa.load(test_batch[i], sr = SR)
                if 'increasing dataset' in test_batch[i].split('/'):
                    data = crop_rec(data)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    data = librosa.amplitude_to_db(data)
                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)

            y_batch = np_utils.to_categorical(y_batch, len(target_names))

            yield x_batch, y_batch

# Splitting into Train/Val/Test

In [6]:
y = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_names, y, 
                                                    test_size=0.10, 
                                                    shuffle=True, 
                                                    random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=0)
print("Train shape: \t{}, \nTest shape: \t{}, \nValid shape: \t{}".format(len(X_train), len(X_test), len(X_val)))

Train shape: 	201287, 
Test shape: 	27957, 
Valid shape: 	50322


In [7]:
callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger(logfile)]

img_input = Input(shape = input_shape)

model = current_model(input_tensor = img_input, classes = len(target_names), weights = None)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

callbacks_list = [ModelCheckpoint(monitor = monitor,
                                filepath = top_weights_path,
                                save_best_only = True,
                                save_weights_only = True,
                                verbose = 1),
                    EarlyStopping(monitor = monitor,
                                patience = es_patience,
                                verbose = 1),
                    ReduceLROnPlateau(monitor = monitor,
                                factor = 0.1,
                                patience = rlr_patience,
                                verbose = 1),
                    CSVLogger(filename = logfile)]

In [8]:
model.fit_generator(train_generator(X_train,
                                    y_train, 
                                    batch_size=32, 
                                    target_names=target_names),
                    steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                    epochs=100, 
                    validation_data = valid_generator(X_val,
                                                      y_val, 
                                                      batch_size=32, 
                                                      target_names=target_names), 
                    validation_steps = int(math.ceil(float(len(X_test)) / float(batch_size))),
                    callbacks = callbacks_list)

Epoch 1/100
 160/6291 [..............................] - ETA: 1:00:28 - loss: 0.9414 - acc: 0.6594

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kalfasyan/miniconda3/envs/wingbeat2/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-f070a00c842d>", line 12, in <module>
    callbacks = callbacks_list)
  File "/home/kalfasyan/miniconda3/envs/wingbeat2/lib/python3.6/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/home/kalfasyan/miniconda3/envs/wingbeat2/lib/python3.6/site-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/home/kalfasyan/miniconda3/envs/wingbeat2/lib/python3.6/site-packages/keras/engine/training_generator.py", line 217, in fit_generator
    class_weight=class_weight)
  File "/home/kalfasyan/miniconda3/envs/wingbeat2/lib/python3.6/site-packages/keras/engine/training.py", line 1217, in train_on_batch
    outputs = self.train_function(ins)
  File "/

KeyboardInterrupt: 

In [ ]:
model.load_weights(top_weights_path)

loss, acc = model.evaluate_generator(valid_generator(X_val, y_val, batch_size=32),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))
#print('loss', loss)
print('Test accuracy:', acc)

In [ ]:
from keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + ".yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
model.save_weights(TEMP_DATADIR + model_name + "_weights.h5")